In [16]:
st=151.03   #current price
K=165  #Strike price
r=4.25    
t= 33
v=20     
coupon = 0.53
r=r-coupon
ttm=t/365

In [17]:
bsm=op.black_scholes(K=K, St=st, r=r, t=t, v=v, type='c')
bsm['Carry Rho']=bsm['greeks']['delta']*ttm*165
bsm

{'value': {'option value': 0.3359599437090406,
  'intrinsic value': 0,
  'time value': 0.3359599437090406},
 'greeks': {'delta': 0.08301107089626869,
  'gamma': 0.016830979206204362,
  'theta': -0.02227999433394731,
  'vega': 0.06942036604441162,
  'rho': 0.011031223810791666},
 'Carry Rho': 1.2383432356991317}

In [18]:
bsm=op.black_scholes(K=K, St=st, r=r, t=t, v=v, type='p')
bsm['Carry Rho']=bsm['greeks']['delta']*ttm*-165
bsm

{'value': {'option value': 13.751949648187235,
  'intrinsic value': 13.969999999999999,
  'time value': -0.21805035181276367},
 'greeks': {'delta': -0.9169889291037313,
  'gamma': 0.016830979206204362,
  'theta': -0.005520019492833374,
  'vega': 0.06942036604441162,
  'rho': -0.1376459723603804},
 'Carry Rho': 13.679464983478951}

In [19]:
def black_scholes_call(S, K, r, q, sigma, T):
    d1 = (np.log(S/K) + (r - q + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    return S*np.exp(-q*T)*N_d1 - K*np.exp(-r*T)*N_d2
def black_scholes_put(S, K, r, q, sigma, T):
    d1 = (np.log(S/K) + (r - q + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    N_minus_d1 = norm.cdf(-d1)
    N_minus_d2 = norm.cdf(-d2)
    return K*np.exp(-r*T)*N_minus_d2 - S*np.exp(-q*T)*N_minus_d1

In [20]:
def call_calculation(S,K,r,q,ivol,T):
    d1 = (np.log(S/K) + (r + (ivol**2)/2)*T)/(ivol*np.sqrt(T))
    d2 = d1 - ivol*np.sqrt(T)
    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    delta_closed = Nd1
    gamma_closed = norm.pdf(d1)/(S*ivol*np.sqrt(T))
    theta_closed = -(S*ivol*norm.pdf(d1))/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*Nd2
    vega_closed = S*np.sqrt(T)*norm.pdf(d1)
    h = 0.01
    S_plus_h = S + h
    S_minus_h = S - h
    V_plus_h = ivol + h 
    V_minus_h = ivol - h 
    R_plus_h = r+h
    R_minus_h = r - h
    C_plus_h = black_scholes_call(S_plus_h, K, r, q, ivol, T)
    C_minus_h = black_scholes_call(S_minus_h, K, r, q, ivol, T)
    delta_finite_diff = (C_plus_h - C_minus_h)/(2*h)
    gamma_finite_diff = (black_scholes_call(S_plus_h, K, r, q, ivol, T) - 2*black_scholes_call(S, K, r, q, ivol, T) + black_scholes_call(S_minus_h, K, r, q, ivol, T))/(h**2)
    theta_finite_diff = -(black_scholes_call(S_plus_h, K, r, q, ivol, T) - black_scholes_call(S_minus_h, K, r, q, ivol, T))/(2*h) - r*black_scholes_call(S, K, r, q, ivol, T)
    #vega_finite_diff = (black_scholes_call(S, K, r, q, V_plus_h, T)-black_scholes_call(S, K, r, q, V_minus_h, T))/(2*h*ivol)
    #rho_finite_diff = (black_scholes_call(S, K, R_plus_h, q, ivol, T)-black_scholes_call(S, K, R_minus_h, q, ivol, T))/(2*h*r)
    return delta_finite_diff,gamma_finite_diff,theta_finite_diff

In [21]:
def put_calculation(S,K,r,q,ivol,T):
    d1 = (np.log(S/K) + (r + (ivol**2)/2)*T)/(ivol*np.sqrt(T))
    d2 = d1 - ivol*np.sqrt(T)
    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    delta_closed = Nd1
    gamma_closed = norm.pdf(d1)/(S*ivol*np.sqrt(T))
    theta_closed = -(S*ivol*norm.pdf(d1))/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*Nd2
    vega_closed = S*np.sqrt(T)*norm.pdf(d1)
    h = 0.01
    S_plus_h = S + h
    S_minus_h = S - h
    V_plus_h = ivol + h 
    V_minus_h = ivol - h 
    R_plus_h = r+h
    R_minus_h = r - h
    C_plus_h = black_scholes_put(S_plus_h, K, r, q, ivol, T)
    C_minus_h = black_scholes_put(S_minus_h, K, r, q, ivol, T)
    delta_finite_diff = (C_plus_h - C_minus_h)/(2*h)
    gamma_finite_diff = (black_scholes_put(S_plus_h, K, r, q, ivol, T) - 2*black_scholes_put(S, K, r, q, ivol, T) + black_scholes_put(S_minus_h, K, r, q, ivol, T))/(h**2)
    theta_finite_diff = -(black_scholes_put(S_plus_h, K, r, q, ivol, T) - black_scholes_put(S_minus_h, K, r, q, ivol, T))/(2*h) - r*black_scholes_put(S, K, r, q, ivol, T)
    #vega_finite_diff = (black_scholes_call(S, K, r, q, V_plus_h, T)-black_scholes_call(S, K, r, q, V_minus_h, T))/(2*h*ivol)
    #rho_finite_diff = (black_scholes_call(S, K, R_plus_h, q, ivol, T)-black_scholes_call(S, K, R_minus_h, q, ivol, T))/(2*h*r)
    return delta_finite_diff,gamma_finite_diff,theta_finite_diff

In [29]:
st=151.03   #current price
K=165  #Strike price
r=0.0425     
t=33      
ivol=0.2     
q = 0.0053
ttm=t/365
x=call_calculation(st,K,r,q,ivol,ttm)
y=put_calculation(st,K,r,q,ivol,ttm)
print('Call ','delta :',x[0],'Gamma :',x[1],'Theta :',x[2])
print('Put ','delta :',y[0],'Gamma :',y[1],'Theta :',y[2])

Call  delta : 0.08297134424122277 Gamma : 0.016822917423553463 Theta : -0.09724280164056233
Put  delta : -0.9165495924605693 Gamma : 0.016822917814351968 Theta : 0.3323717247206667


In [23]:
import math
def combos(n, i):
    return math.factorial(n) / (math.factorial(n-i)*math.factorial(i))

In [24]:
def binom_EU1(S, K , T, r, ivol, N, type_ = 'call'):
    dt = T/N
    u = np.exp(ivol * np.sqrt(dt))
    d = np.exp(-ivol * np.sqrt(dt))
    p = (  np.exp(r*dt) - d )  /  (  u - d )
    value = 0 
    for i in range(N+1):
        node_prob = combos(N, i)*p**i*(1-p)**(N-i)
        ST = S*(u)**i*(d)**(N-i)
        if type_ == 'call':
            value += max(ST-K,0) * node_prob
        elif type_ == 'put':
            value += max(K-ST, 0)*node_prob
        else:
            raise ValueError("type_ must be 'call' or 'put'" )
    
    return value*np.exp(-r*T)


In [25]:
def cal_div_period(T0, Td, T1, N):
    return int((Td - T0).days / (T1 - T0).days * N) 

In [26]:
st = 151.03
K= 165
r = 0.0425-0.0053
ivol = 0.2
T0 = datetime(2022, 3, 13) 
T1= datetime(2022, 4, 15) 
Td = datetime(2022, 4, 11) 
ttm = 33/365
div = 0.88

In [27]:
N = 100
DP = cal_div_period(T0, Td, T1, N)
call= bt_american(True, st, K, ttm, r, 0,ivol, N, dividends=[(DP, div)])
put = bt_american(False, st, K, ttm, r, 0,ivol, N, dividends=[(DP, div)])    

print('value of American call Option w/ div is :',call)
print('value of American put Option w/ div is :',put)

call = bt_american(True, st, K, ttm, r, 0,ivol, N, dividends=[])
put= bt_american(False, st, K, ttm, r, 0,ivol, N, dividends=[])    

print('value of American call Option w/o div is :',call)
print('value of American call Option w/o div is :',put)

value of American call Option w/ div is : 0.29181370999094486
value of American put Option w/ div is : 14.627752573353897
value of American call Option w/o div is : 0.3359668460797688
value of American call Option w/o div is : 14.038608480309883
